In [1]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
# daily revenue
daily = pd.read_csv('../data/customer_daily_revenue.csv', parse_dates=['order_date'])
# rfm
rfm = pd.read_csv('../data/rfm_results.csv')


df = daily.merge(
  rfm[['customer_unique_id', 'rfm_segment_3']],
  on='customer_unique_id',
  how='left'
)

df['rfm_segment_3'] = df['rfm_segment_3'].fillna('Dormant')

In [3]:
df.tail()


,order_date,customer_unique_id,daily_revenue,rfm_segment_3
95583,2018-08-29,7febafa06d9d8f232a900a2937f04338,61.29,Potential
95584,2018-08-29,afbcfd0b9c5233e7ccc73428526fbb52,497.25,VIP
95585,2018-08-29,b4dcade04bc548b7e3b0243c801f8c26,106.95,Potential
95586,2018-08-29,b701bebbdf478f5500348f03aff62121,33.23,Potential
95587,2018-08-29,f80013faf776e37bcea7634d59c2181e,74.21,Potential


# Generating time series

In [4]:
# Times series for total revenue
ts_all = (
  df.groupby('order_date', as_index=False)['daily_revenue']
  .sum()
  .rename(columns={'order_date': 'ds', 'daily_revenue': 'y'})
)

# Time series for each segment
def make_segment_ts(segment):
  return(
    df[df['rfm_segment_3'] == segment]
    .groupby('order_date', as_index=False)['daily_revenue']
    .sum()
    .rename(columns={'order_date': 'ds', 'daily_revenue':'y'})
  )

ts_vip = make_segment_ts('VIP')
ts_potential = make_segment_ts('Potential')

Dormant customers were excluded from forecasting due to sparse and irregular purchasing patterns; they are better addressed via targeted reactivation campaigns rather than trend-based prediction.

In [5]:
def clean_ts(ts):
  # Find the first date that has revenue
  first = ts.loc[ts['y'] > 0, 'ds'].min()
  ts = ts[ts['ds'] >= first].copy()

  #Ensure date continuity
  ts = (
    ts.set_index('ds')
      .asfreq('D', fill_value=0)
      .reset_index()
  )
  return ts

In [6]:
clean_ts_all = clean_ts(ts_all)
clean_ts_vip = clean_ts(ts_vip)
clean_ts_potential = clean_ts(ts_potential)
clean_ts_all.head(), clean_ts_vip.head(), clean_ts_potential.head()

(          ds           y
 0 2016-10-03   559.53000
 1 2016-10-04  9821.41998
 2 2016-10-05  7209.50000
 3 2016-10-06  6733.52000
 4 2016-10-07  6832.76000,
           ds       y
 0 2016-10-03   45.46
 1 2016-10-04    0.00
 2 2016-10-05  140.83
 3 2016-10-06   61.06
 4 2016-10-07    0.00,
           ds      y
 0 2017-03-20  28.62
 1 2017-03-21   0.00
 2 2017-03-22   0.00
 3 2017-03-23   0.00
 4 2017-03-24   0.00)

Missing calendar dates were explicitly filled with zero revenue to ensure a continuous daily time series for seasonality modeling.

In [7]:
# Sanity check
check = ts_vip.merge(ts_potential, on="ds", how="outer", suffixes=("_vip", "_pot")) \
              .merge(ts_all.rename(columns={'y': 'y_total'}), on='ds', how='outer') \
              .fillna(0)

check["sum_segments"] = check["y_vip"] + check["y_pot"]
check.head()

,ds,y_vip,y_pot,y_total,sum_segments
0,2016-09-15,0.00,0.0,0.00000,0.00
1,2016-10-03,45.46,0.0,559.53000,45.46
2,2016-10-04,0.00,0.0,9821.41998,0.00
3,2016-10-05,140.83,0.0,7209.50000,140.83
4,2016-10-06,61.06,0.0,6733.52000,61.06


A separate sanity check was conducted to validate aggregation consistency across segments; however, forecasting models were trained on segment-specific time series to avoid distortion from forced zero-padding

In [8]:
def run_prophet(ts, periods=30):
  m = Prophet(
    weekly_seasonality=True,
    yearly_seasonality=True,
    daily_seasonality=False
  )
  m.fit(ts)
  future = m.make_future_dataframe(periods=periods)
  forecast = m.predict(future)
  
  # fig = m.plot(forecast)
  # plt.show()
  return forecast

# Model Validation

I trained a Prophet model capturing weekly and yearly seasonality, then generated a 30-day future dataframe to forecast segment-level revenue with uncertainty intervals.

In [9]:
#Split the data into train/test
def train_test_split_ts(ts, test_days=30):
  ts = ts.sort_values('ds').reset_index(drop=True)
  cutoff = ts['ds'].max() - pd.Timedelta(days=test_days)
  train = ts[ts['ds'] <=cutoff].copy()
  test = ts[ts['ds'] > cutoff].copy()
  return train, test

#Evalutate
def eval_prophet(ts, test_days=30):
  train, test = train_test_split_ts(ts, test_days)

  m= Prophet(weekly_seasonality=True, yearly_seasonality=True)
  m.fit(train)
  future = m.make_future_dataframe(periods=test_days)
  fc = m.predict(future)

  fc_test = (
    fc.loc[
      fc['ds'].isin(test['ds']),
      ['ds','yhat']
    ].merge(test, on='ds', how = 'inner')
  )

  mae = mean_absolute_error(fc_test['y'],fc_test['yhat'])
  mape = mean_absolute_percentage_error(fc_test['y'], fc_test['yhat'])

  return{'mean_absolute_error': mae, 'mean_absolute_percentage_error': mape}

In [10]:
eval_all = eval_prophet(ts_all)
eval_vip = eval_prophet(ts_vip)
eval_pot = eval_prophet(ts_potential)

eval_all, eval_vip, eval_pot


00:42:58 - cmdstanpy - INFO - Chain [1] start processing
00:42:58 - cmdstanpy - INFO - Chain [1] done processing
00:42:58 - cmdstanpy - INFO - Chain [1] start processing
00:42:58 - cmdstanpy - INFO - Chain [1] done processing
00:42:58 - cmdstanpy - INFO - Chain [1] start processing
00:42:58 - cmdstanpy - INFO - Chain [1] done processing


({'mean_absolute_error': 13374.682032192484,
  'mean_absolute_percentage_error': 1.5629940183199538},
 {'mean_absolute_error': 6433.468844913715,
  'mean_absolute_percentage_error': 2.3280326496223274},
 {'mean_absolute_error': 8850.586954622862,
  'mean_absolute_percentage_error': 1.301918796633552})

In [11]:
fc_all = run_prophet(ts_all)
fc_vip = run_prophet(ts_vip)
fc_pot = run_prophet(ts_potential)

00:42:58 - cmdstanpy - INFO - Chain [1] start processing
00:42:58 - cmdstanpy - INFO - Chain [1] done processing
00:42:58 - cmdstanpy - INFO - Chain [1] start processing
00:42:58 - cmdstanpy - INFO - Chain [1] done processing
00:42:58 - cmdstanpy - INFO - Chain [1] start processing
00:42:58 - cmdstanpy - INFO - Chain [1] done processing


In [12]:
# Get the predicted value for the last 30 days
def forecast_30d_sum(fc):
    return fc.tail(30)['yhat'].sum()

# summary = pd.DataFrame({
#     'segment': ['All', 'VIP', 'Potential'],
#     'forecast_30d_revenue': [
        
#     ]
# })

forecast_30d_sum(fc_all)

993206.8100472785

In [ ]:
actual_recent = ts_all[ts_all['ds'] >= ts_all['ds'].max() - pd.Timedelta(days=90)]
fc_all_30 = fc_all[fc_all['ds'] > ts_all['ds'].max()]
# fc_all_30[['ds', 'yhat']][['ds', 'yhat']].to_csv("../data/forecast30.csv", index=False)

In [14]:
ts_all_90 = ts_all[ts_all['ds'] >= ts_all['ds'].max() - pd.Timedelta(days=90)].rename(columns={'y': 'All'})
ts_vip_90 = ts_vip[ts_vip['ds'] >= ts_vip['ds'].max() - pd.Timedelta(days=90)].rename(columns={'y': 'Vip'})
ts_pot_90 = ts_potential[ts_potential['ds'] >= ts_potential['ds'].max() - pd.Timedelta(days=90)].rename(columns={'y': 'Pot'})

ts_recent_90 = ts_all_90.merge(
  ts_vip_90[['ds', 'Vip']],
  on='ds',
  how = 'left'
).merge(
  ts_pot_90[['ds', 'Pot']],
  on='ds',
  how = 'left'
).assign(period='Actual')

ts_recent_90.head()

,ds,All,Vip,Pot,period
0,2018-05-31,27373.260005,12338.320000,15034.940005,Actual
1,2018-06-01,37284.049988,17535.169988,19748.880000,Actual
2,2018-06-02,28870.669990,12496.909990,16373.760000,Actual
3,2018-06-03,32375.809994,15205.359995,17170.449999,Actual
4,2018-06-04,33866.169990,15401.720000,18464.449990,Actual


In [15]:
fc_all_30 = fc_all.loc[fc_all['ds'] > ts_recent_90['ds'].max(), ['ds', 'yhat']].rename(columns={'yhat': 'All'})
fc_vip_30 = fc_vip.loc[fc_vip['ds'] > ts_recent_90['ds'].max(), ['ds', 'yhat']].rename(columns={'yhat': 'Vip'})
fc_pot_30 = fc_pot.loc[fc_pot['ds'] > ts_recent_90['ds'].max(), ['ds', 'yhat']].rename(columns={'yhat': 'Pot'})

fc_30 = fc_all_30.merge(
  fc_vip_30[['ds', 'Vip']],
  on='ds',
  how = 'left',
).merge(
  fc_pot_30[['ds', 'Pot']],
  on='ds',
  how = 'left'
).assign(period='Forecast')

fc_30.head()

,ds,All,Vip,Pot,period
0,2018-08-30,30013.918515,11825.343070,9586.508411,Forecast
1,2018-08-31,29761.013406,11074.992915,7473.293898,Forecast
2,2018-09-01,24143.279596,9898.098192,3167.165041,Forecast
3,2018-09-02,25765.629973,10310.514131,5078.034822,Forecast
4,2018-09-03,34508.430948,12736.553254,10431.930087,Forecast


In [16]:
ts_mix = pd.concat([ts_recent_90, fc_30], ignore_index=True)


ts_final = (
    ts_mix
      .melt(
          id_vars=["ds", "period"],
          value_vars=["All", "Vip", "Pot"],
          var_name="segment",
          value_name="value"
      )
)

ts_final["value"] = ts_final["value"].clip(lower=0)
ts_final = ts_final.sort_values(
    ["segment", "period", "ds"]
).reset_index(drop=True)

ts_final.to_csv("../data/tableau/ts_final.csv", index=False)